In [2]:
import gc

import numpy as np
import pandas as pd
from scipy import sparse

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import StratifiedShuffleSplit
s
def read_file(file_name, max_iters=float('Inf')):
    data = []
    n_items = 0

    with open(file_name, 'r') as f:
        for row in f:
            n_items += 1
            data.append(row)
            
            if n_items > max_iters:
                break
    return data

# Алгоритм формирования данных

Из подготовленных текстовых файлов необходимо получить разреженные матрицы с помощью:
- TfidfVectorizer для counters
- CountVectorizer для остальных

Полученные данные сохраняются отдельно для каждой категории признаков в npz. Для данных датасета (dataset) - train:
- sp_train_counter.npz
- sp_train_cat.npz
- sp_train_day.npz
- sp_train_day_cat.npz

Аналогично для тестовой выборки:
- sp_test_counter.npz
- sp_test_cat.npz
- sp_test_day.npz
- sp_test_day_cat.npz


##### Дополнительно создаются новые признаки комбинацией cat_feature и day_diff
Полученные файлы:
- sp_train_day_cat.npz
- sp_test_day_cat.npz


#### Все данные для train и test объединяются
После онии используются при обучении финальной модели:
- sp_train_dataset.npz
- sp_test_dataset.npz

#### Создание данных для валидации моделей
Данные из файла sp_train_dataset.npz перемешиваются и разделяется с помощью StratifiedShuffleSplit:
- sp_x_train_shuffled.npz
- sp_x_valid_shuffled.npz
- sp_y_train_shuffled.npz
- sp_y_valid_shuffled.npz

### Counters

In [13]:
%%time
data = read_file('..\\sparse\\dataset_s_counter.csv')
data = [x.rstrip('\n') for x in data]

transformer = TfidfVectorizer(min_df=3, max_df=0.4, sublinear_tf=True)

data_out = transformer.fit_transform(data)
sparse.save_npz('..\\sparse\\sp_train_counter', data_out)
print(data_out.__str__)

del data, data_out
gc.collect()


# Test
data = read_file('..\\sparse\\dataset_s_test_counter.csv')
data = [x.rstrip('\n') for x in data]

data_out = transformer.transform(data)
sparse.save_npz('..\\sparse\\sp_test_counter', data_out)
print(data_out.__str__)

del data, data_out
gc.collect()

<bound method spmatrix.__str__ of <427994x793343 sparse matrix of type '<class 'numpy.float64'>'
	with 279184953 stored elements in Compressed Sparse Row format>>


NameError: name 'parse' is not defined

### cat_feature

In [3]:
%%time
data = read_file('..\\sparse\\dataset_s_cat.csv')

# Удалить пробелы и спец символы
data = [x.replace(' ', '').rstrip('\n') for x in data]

transformer = CountVectorizer(analyzer='char')
# transformer = TfidfVectorizer(analyzer='char', use_idf=False)
data_out = transformer.fit_transform(data)

sparse.save_npz('..\\sparse\\sp_train_cat', data_out)
print(data_out.__str__)


# Test
data = read_file('..\\sparse\\dataset_s_test_cat.csv')

# Удалить пробелы и спец символы
data = [x.replace(' ', '').rstrip('\n') for x in data]
data_out = transformer.transform(data)

sparse.save_npz('..\\sparse\\sp_test_cat', data_out)
print(data_out.__str__)

<bound method spmatrix.__str__ of <427994x6 sparse matrix of type '<class 'numpy.int64'>'
	with 514272 stored elements in Compressed Sparse Row format>>
<bound method spmatrix.__str__ of <181024x6 sparse matrix of type '<class 'numpy.int64'>'
	with 236723 stored elements in Compressed Sparse Row format>>
Wall time: 6.15 s


### diff_day

In [4]:
%%time
data = read_file('..\\sparse\\dataset_s_day.csv')

# Удалить спец символы
data = [x.rstrip('\n') for x in data]

transformer = CountVectorizer()
# transformer = TfidfVectorizer(use_idf=False)
data_out = transformer.fit_transform(data)
print(data_out.__str__)

sparse.save_npz('..\\sparse\\sp_train_day', data_out)


# Test
data = read_file('..\\sparse\\dataset_s_test_day.csv')

# Удалить спец символы
data = [x.rstrip('\n') for x in data]

data_out = transformer.transform(data)
print(data_out.__str__)

sparse.save_npz('..\\sparse\\sp_test_day', data_out)

<bound method spmatrix.__str__ of <427994x51 sparse matrix of type '<class 'numpy.int64'>'
	with 9683247 stored elements in Compressed Sparse Row format>>
<bound method spmatrix.__str__ of <181024x51 sparse matrix of type '<class 'numpy.int64'>'
	with 4701874 stored elements in Compressed Sparse Row format>>
Wall time: 15.8 s


# Создание доп признаков вида day57_cat1

Дополнительные признаки получены путем комбинации номеров дней и категориального признака cat_feature

In [5]:
# Для dataset

cat_f = pd.read_csv('..\\sparse\\dataset_s_cat.csv', sep='\t', header=None)[0].tolist()
days = pd.read_csv('..\\sparse\\dataset_s_day.csv', sep='\t', header=None)[0].tolist()

new_feat = []
for i in range(len(cat_f)):
    new_feat.append(' '.join([f'day{d}_cat{c}' for d,c in zip(days[i].split(), cat_f[i].split())]))
    
pd.DataFrame(new_feat).to_csv('..\\sparse\\dataset_s_day_cat.csv', header=None, index=None)

In [6]:
# Для dataset_test

cat_f = pd.read_csv('..\\sparse\\dataset_s_test_cat.csv', sep='\t', header=None)[0].tolist()
days = pd.read_csv('..\\sparse\\dataset_s_test_day.csv', sep='\t', header=None)[0].tolist()

new_feat = []
for i in range(len(cat_f)):
    new_feat.append(' '.join([f'day{d}_cat{c}' for d,c in zip(days[i].split(), cat_f[i].split())]))

pd.DataFrame(new_feat).to_csv('..\\sparse\\dataset_s_test_day_cat.csv', header=None, index=None)

In [8]:
%%time

# Датасет
data = read_file('..\\sparse\\dataset_s_day_cat.csv')
data = [x.rstrip('\n') for x in data]

transformer = CountVectorizer()
# transformer = TfidfVectorizer(use_idf=False)
data_out = transformer.fit_transform(data)

sparse.save_npz('..\\sparse\\sp_train_day_cat', data_out)


# Тестовая выборка
data = read_file('..\\sparse\\dataset_s_test_day_cat.csv')
data = [x.rstrip('\n') for x in data]

data_out = transformer.transform(data)
sparse.save_npz('..\\sparse\\sp_test_day_cat', data_out)

Wall time: 21.7 s


## Объединение данных
### train

In [15]:
sp_counter = sparse.load_npz('..\\sparse\\sp_train_counter.npz')
sp_cat = sparse.load_npz('..\\sparse\\sp_train_cat.npz')
sp_day = sparse.load_npz('..\\sparse\\sp_train_day.npz')
sp_day_cat = sparse.load_npz('..\\sparse\\sp_train_day_cat.npz')

# Объединить данные
X = sparse.hstack([sp_counter, sp_cat, sp_day, sp_day_cat])
print(X.__str__)


# Сохранить данные
sparse.save_npz('..\\sparse\\sp_train_dataset', X)

del sp_counter, sp_cat, sp_day, sp_day_cat, X
gc.collect()

<bound method spmatrix.__str__ of <427994x793760 sparse matrix of type '<class 'numpy.float64'>'
	with 302256817 stored elements in COOrdinate format>>


60

### test

In [16]:
sp_test_counter = sparse.load_npz('..\\sparse\\sp_test_counter.npz')
sp_test_cat = sparse.load_npz('..\\sparse\\sp_test_cat.npz')
sp_test_day = sparse.load_npz('..\\sparse\\sp_test_day.npz')
sp_test_day_cat = sparse.load_npz('..\\sparse\\sp_test_day_cat.npz')

# Объединить данные
X = sparse.hstack([sp_test_counter, sp_test_cat, sp_test_day, sp_test_day_cat])
print(X.__str__)

sparse.save_npz('..\\sparse\\sp_test_dataset', X)

del sp_test_counter, sp_test_cat, sp_test_day, sp_test_day_cat, X
gc.collect()

<bound method spmatrix.__str__ of <181024x793760 sparse matrix of type '<class 'numpy.float64'>'
	with 196816553 stored elements in COOrdinate format>>


60

# Перемешать и разделить train_dataset на train / valid

Перемешать полученные данные и разделить их на тренировочную и валидационную выборку для дальнейшего подбора параметров моделей на валидации.

Полученные файлы:
- sp_x_train_shuffled.npz
- sp_x_valid_shuffled.npz
- sp_y_train_shuffled.npz
- sp_y_valid_shuffled.npz

In [17]:
# Датасет
X_train = sparse.load_npz('..\\sparse\\sp_train_dataset.npz').tocsr()

# Правильные ответы на train
y_train = read_file('..\\sparse\\dataset_s_labels.csv')
y_train = np.array([int(a) for a in y_train])

X_train, y_train.shape

(<427994x793760 sparse matrix of type '<class 'numpy.float64'>'
 	with 302256817 stored elements in Compressed Sparse Row format>, (427994,))

In [18]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.10, random_state=17)

for train_index, valid_index in sss.split(X_train, y_train):
    print("TRAIN:", train_index, "TEST:", valid_index)
    print("TRAIN len:", len(train_index), "TEST len:", len(valid_index))

TRAIN: [ 55581 260848  35002 ... 292783 268123 310387] TEST: [ 23716  75326 408376 ... 102404 316201  86044]
TRAIN len: 385194 TEST len: 42800


In [19]:
sparse.save_npz('..\\sparse\\sp_x_train_shuffled', X_train[train_index])
sparse.save_npz('..\\sparse\\sp_x_valid_shuffled', X_train[valid_index])

sparse.save_npz('..\\sparse\\sp_y_train_shuffled', sparse.csr_matrix(y_train[train_index]))
sparse.save_npz('..\\sparse\\sp_y_valid_shuffled', sparse.csr_matrix(y_train[valid_index]))

In [20]:
del X_train, y_train
gc.collect()

15